In [1]:
import sys
import glob
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('../readers')
from loadGhGraphs import loadGhGraphs
from altTrussViz import plotTruss
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'gcnSurrogateUtil'

In [ ]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphs = loadGhGraphs(doeFile, NUM_DV=5)

In [ ]:
plotTruss(allGraphs[0])

In [ ]:
pred = allGraphs[0].y+0.1*np.random.rand()
plotTruss(allGraphs[0], showDeformed=True, defScale=30, showUndeformed=True, prediction=pred.numpy())

## Visualize the design space

In [ ]:
alt.themes.enable('none')

In [ ]:
doeFile = "C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/param_0_design_9_N_5.csv"
allGraphs = loadGhGraphs(doeFile, NUM_DV=1)
opacities = np.linspace(0.1,0.6,5)
opacities[-1] = 1
plotList = [plotTruss(g, withoutConfigure=True, lineOpacity=o) for g,o in zip(allGraphs, opacities)]
alt.layer(*plotList, background='none').configure_view(strokeOpacity=0)

In [ ]:
doeFile = "C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/param_1_design_9_N_5.csv"
allGraphs = loadGhGraphs(doeFile, NUM_DV=1)
plotList = [plotTruss(g, withoutConfigure=True, lineOpacity=o) for g,o in zip(allGraphs, opacities)]
alt.layer(*plotList, background='none').configure_view(strokeOpacity=0)

In [ ]:
doeFile = "C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/param_2_design_9_N_5.csv"
allGraphs = loadGhGraphs(doeFile, NUM_DV=1)
plotList = [plotTruss(g, withoutConfigure=True, lineOpacity=o) for g,o in zip(allGraphs, opacities)]
alt.layer(*plotList, background='none').configure_view(strokeOpacity=0)

In [ ]:
doeFile = "C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/param_3_design_9_N_5.csv"
allGraphs = loadGhGraphs(doeFile, NUM_DV=1)
plotList = [plotTruss(g, withoutConfigure=True, lineOpacity=o) for g,o in zip(allGraphs, opacities)]
alt.layer(*plotList, background='none').configure_view(strokeOpacity=0)

In [ ]:
doeFile = "C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/param_4_design_9_N_5.csv"
allGraphs = loadGhGraphs(doeFile, NUM_DV=1)
plotList = [plotTruss(g, withoutConfigure=True, lineOpacity=o, background='none') for g,o in zip(allGraphs, opacities)]
alt.layer(*plotList, background='none').configure_view(strokeOpacity=0)

## Visualize one design

In [ ]:
doeFile = "C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/param_0_design_9_N_5.csv"
allGraphs = loadGhGraphs(doeFile, NUM_DV=1)
plotTruss(allGraphs[0], background='none', width=150, height=75, z=7, showPoints=True, pointSize=50)

In [ ]:
plotTruss(allGraphs[0], showDeformed=True, defScale=80, background='none', width=150, height=75, z=7, prediction=allGraphs[0].y*(1+1.5*(np.random.rand()-0.5)))

## Visualize a few designs

In [ ]:
def filterbyDisp(graphList, pctCutoff):
    maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
    df = pd.DataFrame(maxes, columns=['maxes'])
    mask = df['maxes'].rank(pct=True) <= pctCutoff
    return [g for g,b in zip(graphList, mask) if b]

def partitionGraphList(allGraphs):
    allIds = list(range(len(allGraphs)))
    other, testIds = train_test_split(allIds, test_size=0.2, shuffle=True, random_state=1234) # 20% test
    trainIds, valIds = train_test_split(other, test_size=0.15, shuffle=True, random_state=1234) # 15% val 15% test
    trainGraphs = [allGraphs[i] for i in trainIds]
    valGraphs = [allGraphs[i] for i in valIds]
    testGraphs = [allGraphs[i] for i in testIds]
    return trainGraphs, valGraphs, testGraphs

doeFile = "C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/design_9_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainGraphs, valGraphs, testGraphs = partitionGraphList(allGraphs)

In [ ]:
plotList = [plotTruss(g, height=215, width=500, lineWidth=3, z=7, withoutConfigure=True) for g in testGraphs[:36]]
columnList = [alt.vconcat(*plotList[6*i:6*(i+1)], spacing=0) for i in range(6)]
alt.hconcat(*columnList, spacing=0, background='none').configure_view(strokeOpacity=0)

## Visualize several topologies

In [ ]:
trainSets ,valSets ,testSets = {}, {}, {}

doeFiles = np.sort(glob.glob("C:/Users/ewhalen/OneDrive - Massachusetts Institute of Technology/research/data/trusses/EW/v1.3/design*.csv"))
for doeFile in doeFiles:
    designName = doeFile.split('\\')[-1].split('_N')[0]
    print(f'loading {designName}')
    allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)
    allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
    trainData, valData, testData = partitionGraphList(allGraphs)
    trainSets[designName] = trainData
    valSets[designName] = valData
    testSets[designName] = testData

In [ ]:
columnList = []
for groupName, testSet in testSets.items():
    plotList = []
    for graph in testSet[:5]:
        plotList.append(plotTruss(graph, height=215, width=500, lineWidth=3, z=7, withoutConfigure=True, showPoints=True, pointSize=150))
    col = alt.vconcat(*plotList, spacing=0)
    columnList.append(col)
alt.hconcat(*columnList, spacing=0).configure_view(strokeOpacity=0)